# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/defender_qt.pickle','rb') as f:
    defender_qt = pickle.load(f)

defender_model = load_model('./models/defender_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
defender_df = premier_df[premier_df['포지션']=='수비수']

In [5]:
# 0골인 선수 보정
midfielder_df['골'] = midfielder_df['골']+1
midfielder_df['경기당 슈팅/90'] = midfielder_df['경기당 슈팅/90']+0.01

# Shoot Accuracy Weighted Goal
midfielder_df['SAWG'] = (midfielder_df['경기당 유효 슈팅/90'] / midfielder_df['경기당 슈팅/90'])*midfielder_df['골']

In [8]:
# 0도움인 선수 보정
midfielder_df['도움'] = midfielder_df['도움']+1

# Pass Accuracy Weighted Assist And Keypass
midfielder_df['PAWAAK'] = (midfielder_df['경기당 패스 성공/90'] / midfielder_df['경기당 패스 시도/90'])*midfielder_df['도움']+midfielder_df['키패스/90']

In [ ]:
midfielder_df['태클 시도/90'] = midfielder_df['태클 성공/90']/midfielder_df['태클 성공률']

In [ ]:
features = ('PAWAAK','SAWG','기회 창출/90','드리블 성공/90','반칙 수/90','크로스 성공률','태클 시도/90','파울 획득')

In [ ]:
midfielder_df.loc[:,features] = midfielder_qt.transform(midfielder_df.loc[:,features])

In [ ]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [9]:
defender_df.loc[:,outlier_features] = defender_qt.transform(defender_df.loc[:,outlier_features])
defender_df.loc[:,nonoutlier_features] = defender_ss.transform(defender_df.loc[:,nonoutlier_features])
defender_df.loc[:,order_features] = defender_mm.transform(defender_df.loc[:,order_features])

In [10]:
defender_df = defender_df.reset_index(drop=True)

In [11]:
good_features = ['경기당 패스 시도/90','경기당 슈팅/90','기회 창출/90','태클 성공/90',
                 '드리블 성공/90','파울 획득','크로스 성공률','태클 성공률','인터셉트/90']

In [12]:
defender_df['예측PPG']=defender_model.predict(tf.constant(defender_df.loc[:,good_features].values))
defender_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
0,Ben Gibson,2021-2022,수비수,노리치,잉글랜드 프리미어 리그,0.250,0.000000,0,8,12,...,0.944414,0.194124,0.325164,-0.690633,-0.338888,-0.757117,-0.449904,-1.741291,0.95,1.314805
1,Ozan Kabak,2021-2022,수비수,노리치,잉글랜드 프리미어 리그,0.000,0.071429,21/7/5 - 22/5/31,7,13,...,0.173050,1.256109,1.246317,0.025581,0.154074,-1.465234,-0.150983,-0.565949,0.94,1.620615
2,Diego Carlos,2021-2022,수비수,뉴캐슬,잉글랜드 프리미어 리그,0.125,0.071429,0,9,3,...,-0.708508,1.737936,1.775040,-1.824827,0.102491,0.128240,0.089938,-5.199338,1.76,1.858013
3,Jamaal Lascelles,2021-2022,수비수,뉴캐슬,잉글랜드 프리미어 리그,0.125,0.071429,0,19,6,...,-1.259482,-0.275411,-0.201209,-1.394173,-1.270008,-0.154074,0.674396,-0.366106,1.75,1.226763
4,Wilfred Ndidi,2021-2022,수비수,레스터,잉글랜드 프리미어 리그,0.500,0.214286,0,14,7,...,-1.810456,-0.233620,-0.285804,0.997819,0.979704,-0.025581,-2.198817,0.723818,1.48,1.683097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Japhet Tanganga,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.125,0.142857,0,18,8,...,-0.267729,-1.430812,-1.709831,1.109677,0.876143,1.635039,-0.463289,1.519367,1.88,1.677509
82,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.000,0.071429,0,17,9,...,-2.802210,1.130736,1.180520,-0.596232,-0.901454,1.289343,0.727935,-0.863696,2.00,1.895835
83,Eric Dier,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.000,0.214286,0,12,5,...,1.495388,0.142500,0.160673,0.586078,1.117332,0.051178,0.072092,1.031476,1.86,1.534523
84,Ben Davies,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.000,0.214286,0,11,3,...,0.503635,-2.313341,-3.183205,5.199338,2.116159,1.495846,-1.467129,1.992099,2.12,1.337747


In [13]:
defender_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
82,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.000,0.071429,0,17,9,...,-2.802210,1.130736,1.180520,-0.596232,-0.901454,1.289343,0.727935,-0.863696,2.00,1.895835
17,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,0.375,0.214286,0,24,10,...,-1.039093,2.283678,1.993579,1.153848,1.677539,0.393598,-1.458206,-0.627072,1.82,1.891601
12,Fabinho,2021-2022,수비수,리버풀,잉글랜드 프리미어 리그,0.125,0.357143,0,31,11,...,-0.267729,1.226610,0.950232,1.095425,1.092971,0.449514,-1.913280,0.477996,2.08,1.862765
80,William Carvalho,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.125,0.357143,0,10,4,...,-0.818703,2.195180,2.151020,1.289343,0.827130,-0.311919,-2.702968,-1.270008,1.79,1.861998
71,Antonio Rüdiger,2021-2022,수비수,첼시,잉글랜드 프리미어 리그,0.250,0.071429,0,28,7,...,-0.047339,0.902114,0.999580,-0.449514,-0.627072,1.146415,0.473628,0.338888,1.98,1.861596


In [14]:
defender_min = np.min(defender_df['예측PPG'].tolist())
defender_min

1.0806928873062134

In [15]:
defender_df['WAR'] = defender_df['예측PPG'].map(lambda x : x-defender_min+1)

In [16]:
defender_df.sort_values(by='WAR',ascending=False).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
82,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.000,0.071429,0,17,9,...,1.130736,1.180520,-0.596232,-0.901454,1.289343,0.727935,-0.863696,2.00,1.895835,1.815142
17,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,0.375,0.214286,0,24,10,...,2.283678,1.993579,1.153848,1.677539,0.393598,-1.458206,-0.627072,1.82,1.891601,1.810908
12,Fabinho,2021-2022,수비수,리버풀,잉글랜드 프리미어 리그,0.125,0.357143,0,31,11,...,1.226610,0.950232,1.095425,1.092971,0.449514,-1.913280,0.477996,2.08,1.862765,1.782072
80,William Carvalho,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.125,0.357143,0,10,4,...,2.195180,2.151020,1.289343,0.827130,-0.311919,-2.702968,-1.270008,1.79,1.861998,1.781306
71,Antonio Rüdiger,2021-2022,수비수,첼시,잉글랜드 프리미어 리그,0.250,0.071429,0,28,7,...,0.902114,0.999580,-0.449514,-0.627072,1.146415,0.473628,0.338888,1.98,1.861596,1.780903
2,Diego Carlos,2021-2022,수비수,뉴캐슬,잉글랜드 프리미어 리그,0.125,0.071429,0,9,3,...,1.737936,1.775040,-1.824827,0.102491,0.128240,0.089938,-5.199338,1.76,1.858013,1.777320
58,Willy Boly,2021-2022,수비수,울브스,잉글랜드 프리미어 리그,0.125,0.142857,0,7,6,...,1.371649,1.384960,0.180012,-1.329529,0.837824,0.214861,-1.023392,1.00,1.837880,1.757188
45,Douglas Luiz,2021-2022,수비수,아스톤 빌라,잉글랜드 프리미어 리그,0.625,0.071429,0,8,3,...,1.174985,0.245268,1.821545,0.719072,1.329529,-2.475431,0.819226,1.42,1.818272,1.737579
5,Wesley Fofana,2021-2022,수비수,레스터,잉글랜드 프리미어 리그,0.250,0.214286,0,20,13,...,0.882448,0.778691,-0.154074,0.206072,1.093098,0.290706,0.101108,1.46,1.818271,1.737578
20,Aymeric Laporte,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,0.125,0.071429,0,23,10,...,0.557952,0.637698,-0.690633,-0.025581,0.981126,-0.110830,0.101108,1.80,1.791757,1.711064


In [17]:
defender_df.loc[:,outlier_features] = defender_qt.inverse_transform(defender_df.loc[:,outlier_features])
defender_df.loc[:,nonoutlier_features] = defender_ss.inverse_transform(defender_df.loc[:,nonoutlier_features])
defender_df.loc[:,order_features] = defender_mm.inverse_transform(defender_df.loc[:,order_features])

In [18]:
defender_df[['이름','시즌','경기당 패스 시도/90','경기당 슈팅/90','기회 창출/90','태클 성공/90','드리블 성공/90','파울 획득','크로스 성공률','태클 성공률','인터셉트/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,경기당 패스 시도/90,경기당 슈팅/90,기회 창출/90,태클 성공/90,드리블 성공/90,파울 획득,크로스 성공률,태클 성공률,인터셉트/90,WAR
82,Davinson Sánchez,2021-2022,59.65,0.74,0.17,0.68,0.48,6.0,100.0,57.0,1.83,1.815142
17,Rodri,2021-2022,64.34,0.45,0.30,2.43,0.23,17.0,66.0,73.0,0.72,1.810908
12,Fabinho,2021-2022,60.04,0.68,0.22,1.70,0.24,24.0,31.0,80.0,1.28,1.782072
80,William Carvalho,2021-2022,63.98,0.48,0.60,1.45,0.12,12.0,0.0,75.0,0.66,1.781306
71,Antonio Rüdiger,2021-2022,58.72,0.63,0.07,0.74,0.43,20.0,28.0,82.0,2.31,1.780903
2,Diego Carlos,2021-2022,62.12,0.35,0.06,0.94,0.18,2.0,0.0,76.0,1.82,1.777320
58,Willy Boly,2021-2022,60.63,0.45,0.04,0.58,0.32,11.0,50.0,76.0,2.55,1.757188
45,Douglas Luiz,2021-2022,59.83,1.68,0.06,1.33,0.50,14.0,66.0,62.0,0.69,1.737579
5,Wesley Fofana,2021-2022,58.64,0.91,0.06,0.98,0.41,12.0,9.0,90.0,1.75,1.737578
20,Aymeric Laporte,2021-2022,57.32,0.53,0.02,0.90,0.37,13.0,20.0,71.0,2.13,1.711064
